In [ ]:
import os

In [ ]:
import tensorflow as tf
dataset = tf.keras.utils.get_file(
    fname="notMNIST_large.tar.gz", 
    origin="http://commondatastorage.googleapis.com/books1000/notMNIST_large.tar.gz", 
    extract=True
)

In [ ]:
PATH = os.path.dirname(dataset)
root_dir = os.path.join(PATH, "notMNIST_large")

In [ ]:
image_path_list = []
image_label_list = []

for label in os.listdir(root_dir):
    target_dir = os.path.join(root_dir, label)
    for image_filename in os.listdir(target_dir):
        image_path_list.append(
            os.path.join(target_dir, image_filename)
        )
        image_label_list.append(label)

In [ ]:
len(image_path_list), len(image_label_list) 

In [ ]:
from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()
image_label_list = lbl.fit_transform(image_label_list)

In [ ]:
image_path_list[:3]

In [ ]:
image_label_list[:3]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_dev, X_test, y_dev, y_test = train_test_split(
    image_path_list, image_label_list, test_size=0.15,stratify=image_label_list)

In [ ]:
from collections import Counter
Counter(y_test)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_dev, y_dev, test_size=0.20,stratify=y_dev)

In [ ]:
Counter(y_val)

In [ ]:
!python -m pip install Pillow

In [ ]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
for image_path in X_train[:3]:
    display.display(Image.open(str(image_path)))

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
for f in train_dataset.take(3):
    print(f)

In [ ]:
class_names = {k:i for i, k in enumerate(set(y_val))}

def _preprocess_image(image):   
    image = tf.image.decode_png(image, channels=1)   
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [28, 28,])
    image /= 255.0  # normalize to [0,1] range    
    return image

def _load_and_preprocess_image(path):   
    image = tf.io.read_file(path)
    image = _preprocess_image(image)
    return image

def load_and_preprocess_from_path_label(path, label):   
    return _load_and_preprocess_image(path), label  

def get_dataset(ds):
    image_label_ds = ds.map(load_and_preprocess_from_path_label)
#   image_label_ds = image_label_ds.shuffle(1024)
    image_label_ds = image_label_ds.batch(256)
#   image_label_ds = image_label_ds.prefetch(buffer_size=512)
    return image_label_ds

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

train_dataset = get_dataset(train_dataset)
val_dataset = get_dataset(val_dataset)
test_dataset = get_dataset(test_dataset)


In [ ]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten

In [ ]:
model = Sequential()
model.add(Flatten(input_shape = (28, 28,1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=10,
                    verbose=1)

score = model.evaluate(test_dataset, verbose=0)
